# Optical Flow Random Shaking Volume Denoising by Slices using Farneback2d (tomo2_L1G1_bin4)

In [ ]:
local_debug = True

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    from google.colab import drive
    drive.mount('/content/drive')
    !cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    !cp ~/Downloads/{vol_name}.tif .

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
from denoising.volume.OFRSVD import Random_Shaking_Denoising_by_Slices # pip install denoising " @ git+https://github.com/vicente-gonzalez-ruiz/denoising"

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
import skimage.io

In [ ]:
import numpy as np

In [ ]:
import logging
#logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")

In [ ]:
from collections import namedtuple
Args = namedtuple("args", ["input", "output"])
args = Args("/home/vruiz/cryoCARE/tomo2_L1G1_bin4.rec", "denoised")

In [ ]:
import mrcfile
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1I2uIfM00ZNeMjYy4OeZ4hSO-bxpE3oZb"
    gdown https://drive.google.com/uc?id=$FILEID
fi
set +x
# https://drive.google.com/file/d/1I2uIfM00ZNeMjYy4OeZ4hSO-bxpE3oZb/view?usp=sharing

noisy = opticalflow3D.helpers.load_image(args.input)

In [ ]:
np.min(noisy)

In [ ]:
np.max(noisy)

In [ ]:
#noisy = (255*(noisy.astype(np.float32) - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)
noisy = (255*(noisy.astype(np.float32) - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.float32)

In [ ]:
np.min(noisy)

In [ ]:
np.max(noisy)

In [ ]:
#noisy = noisy[230:250, 450:760, 210:510]
#noisy = noisy[180:220, 400:800, 200:600]
noisy = noisy[noisy.shape[0]//2-10:noisy.shape[0]//2+10, :, :]

In [ ]:
noisy.shape

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 16))
axs.imshow(noisy[:, ::-1, :][10], cmap="gray")
axs.set_title(f"Noisy")
fig.tight_layout()
plt.show()

In [ ]:
RS_std_dev = 1.0
N_iters = 25

In [ ]:
denoiser = Random_Shaking_Denoising_by_Slices()
denoised = denoiser.filter_volume(noisy, std_dev=RS_std_dev, N_iters=N_iters)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(32, 64))
#axs[0].imshow(noisy[:, ::-1, :][210][400:800, 200:600], cmap="gray")
axs[0].imshow(noisy[:, ::-1, :][noisy.shape[0]//2], cmap="gray")
axs[0].set_title(f"Noisy")
#axs[1].imshow(denoised[:, ::-1, :][210][400:800, 200:600], cmap="gray")
axs[1].imshow(denoised[:, ::-1, :][noisy.shape[0]//2], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16], denoised[16])})")
fig.tight_layout()
plt.show()

In [ ]:
print(f"{args.output}_{RS_std_dev}_{N_iters}.mrc")
with mrcfile.new(f"{args.output}_{RS_std_dev}_{N_iters}.mrc", overwrite=True) as mrc:
            mrc.set_data(denoised.astype(np.float32))
            mrc.data
skimage.io.imsave(f"{args.output}_{RS_std_dev}_{N_iters}.tif", denoised, imagej=True)